# House Prices Exploration

In this competition, we are trying to figure out the prices for which a bunch of houses have been sold.
A lot of these features can be used out-of-the-box, so let's check what we have.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import sys

# Checking The Data

Let's begin by first loading our training and test set and taking a look at them.

In [ ]:
# Id column won't be useful for building the model,
# so let's make it the index of the dataframe
train = pd.read_csv("input/train.csv", index_col=0)
test = pd.read_csv("input/test.csv", index_col=0)

In [ ]:
train.shape, test.shape

In [ ]:
train.head(5)


Our dataset has 1460 and 1459 houses on our training and test set, respectively. We have a lot of categorial features in this dataset, so it's interesting to break them into dummy variables. To make it easier for us to create and manipulate these variables, let's join the training and test set, so that both will end up with the same columns. We can split them back after we've done all the data cleaning.

Note: model evaluation is based on the log of SalePrice, so we will save it as log too. We need to change it back before exporting our submission file.


## Removing outliers



Before doing anything, we are going to remove outliers from our training data. Later on we are going to merge training and test set to perform transformations, so it's important to deal with outliers before that (we don't want to remove any data from the test set).

 #### Feature: OverralQual

In [ ]:
#for i in range(9):
#    temp = train.loc[train['OverallQual'] == i+1]
    #print(temp["SalePrice"], temp.shape)

#    temp = temp[~(np.abs(temp.SalePrice-temp.SalePrice.mean())<=(2*temp.SalePrice.std()))]
    #print(temp["SalePrice"], temp.shape)

#    train = train[~train.isin(temp)].dropna(how='all')
#    print(i+1, train.shape)

#=======10======
temp = train.loc[train['OverallQual'] == 10]
#print(temp["SalePrice"], temp.shape)

temp = temp[~(np.abs(temp.SalePrice-temp.SalePrice.mean())<=(1*temp.SalePrice.std()))]
#print(temp["SalePrice"], temp.shape)

#print(train.shape)
train = train[~train.isin(temp)].dropna(how='all')
print(10, train.shape)


 #### Feature: OverralCond

In [ ]:
#for i in range(10):
#    temp = train.loc[train['OverallCond'] == i]
#    #print(temp["SalePrice"], temp.shape)

#    temp = temp[~(np.abs(temp.SalePrice-temp.SalePrice.mean())<=(2*temp.SalePrice.std()))]
#    #print(temp["SalePrice"], temp.shape)

#    train = train[~train.isin(temp)].dropna(how='all')
#    print(i, train.shape)

In [ ]:
# let's remove sale prices of our train before joining, and save them for later use
Y_train = np.log(train.pop('SalePrice'))

all_df = pd.concat((train, test), axis=0)
all_df.shape

# Variable Transformations

First thing to notice is that one of the features in the dataset, MSSubClass, is a categorical feature stored as a numeric one.

In [ ]:
all_df['MSSubClass'].dtypes

In [ ]:
all_df['MSSubClass'] = all_df['MSSubClass'].apply(str)
all_df['MSSubClass'].dtypes

These transformations were made by Boris Klyus on a Kaggle notebook. Here he is simply changing all categorical (and some numerical) features that comes with NaN values to a more informative label

In [ ]:
all_df.loc[all_df.Alley.isnull(), 'Alley'] = 'NoAlley'
all_df.loc[all_df.MasVnrType.isnull(), 'MasVnrType'] = 'None' # no good
all_df.loc[all_df.MasVnrType == 'None', 'MasVnrArea'] = 0
all_df.loc[all_df.BsmtQual.isnull(), 'BsmtQual'] = 'NoBsmt'
all_df.loc[all_df.BsmtCond.isnull(), 'BsmtCond'] = 'NoBsmt'
all_df.loc[all_df.BsmtExposure.isnull(), 'BsmtExposure'] = 'NoBsmt'
all_df.loc[all_df.BsmtFinType1.isnull(), 'BsmtFinType1'] = 'NoBsmt'
all_df.loc[all_df.BsmtFinType2.isnull(), 'BsmtFinType2'] = 'NoBsmt'
all_df.loc[all_df.BsmtFinType1=='NoBsmt', 'BsmtFinSF1'] = 0
all_df.loc[all_df.BsmtFinType2=='NoBsmt', 'BsmtFinSF2'] = 0
all_df.loc[all_df.BsmtFinSF1.isnull(), 'BsmtFinSF1'] = all_df.BsmtFinSF1.median()
all_df.loc[all_df.BsmtQual=='NoBsmt', 'BsmtUnfSF'] = 0
all_df.loc[all_df.BsmtUnfSF.isnull(), 'BsmtUnfSF'] = all_df.BsmtUnfSF.median()
all_df.loc[all_df.BsmtQual=='NoBsmt', 'TotalBsmtSF'] = 0
all_df.loc[all_df.FireplaceQu.isnull(), 'FireplaceQu'] = 'NoFireplace'
all_df.loc[all_df.GarageType.isnull(), 'GarageType'] = 'NoGarage'
all_df.loc[all_df.GarageFinish.isnull(), 'GarageFinish'] = 'NoGarage'
all_df.loc[all_df.GarageQual.isnull(), 'GarageQual'] = 'NoGarage'
all_df.loc[all_df.GarageCond.isnull(), 'GarageCond'] = 'NoGarage'
all_df.loc[all_df.BsmtFullBath.isnull(), 'BsmtFullBath'] = 0
all_df.loc[all_df.BsmtHalfBath.isnull(), 'BsmtHalfBath'] = 0
all_df.loc[all_df.KitchenQual.isnull(), 'KitchenQual'] = 'TA'
all_df.loc[all_df.MSZoning.isnull(), 'MSZoning'] = 'RL'
all_df.loc[all_df.Utilities.isnull(), 'Utilities'] = 'AllPub'
all_df.loc[all_df.Exterior1st.isnull(), 'Exterior1st'] = 'VinylSd'
all_df.loc[all_df.Exterior2nd.isnull(), 'Exterior2nd'] = 'VinylSd'
all_df.loc[all_df.Functional.isnull(), 'Functional'] = 'Typ'
all_df.loc[all_df.SaleCondition.isnull(), 'SaleCondition'] = 'Normal'
all_df.loc[all_df.SaleCondition.isnull(), 'SaleType'] = 'WD'
all_df.loc[all_df['PoolQC'].isnull(), 'PoolQC'] = 'NoPool'
all_df.loc[all_df['Fence'].isnull(), 'Fence'] = 'NoFence'
all_df.loc[all_df['MiscFeature'].isnull(), 'MiscFeature'] = 'None'
all_df.loc[all_df['Electrical'].isnull(), 'Electrical'] = 'SBrkr'
# only one is null and it has type Detchd
all_df.loc[all_df['GarageArea'].isnull(), 'GarageArea'] = all_df.loc[all_df['GarageType']=='Detchd', 'GarageArea'].mean()
all_df.loc[all_df['GarageCars'].isnull(), 'GarageCars'] = all_df.loc[all_df['GarageType']=='Detchd', 'GarageCars'].median()

## Creating dummy variables

Now that we have everything together, we need to generate dummy variables for our categorical features. Pandas can help us with the get_dummy_variable function. It generates dummy variables for a series or an entire dataframe. 

In [ ]:
all_dummy_df = pd.get_dummies(all_df)

# let's check if everything went alright
all_dummy_df.head(5)


Column order looks a little messy. They are in alphabetical order now, but that shouldn't be a problem. But we do have a lot of columns, which is possibly a bigger deal to be worried with.

Now that we have our dummy variables, we should check for NaN values in our columns. For the categorical features, pandas' get_dummy_variables takes care of this for us. For example, for a feature X which could get values a, b or c, if an individual has NaN for X, all 3 resulting columns (X_a, X_b, X_c) will be assigned 0. (Note: it can be useful to create a new column for missing values, e.g., X_missing, because not being assigned to any label could be informative; we should get back at this later)

## Missing Values

Since there's no missing values for our categorical features, we just need to check the numerical ones.


In [ ]:
all_dummy_df.isnull().sum().sort_values(ascending=False).head(15)

We need to take care of these missing values. For now, let's just assign the mean for all missing values. There's probably better ways to deal with that, but let's just do the easy way now.

In [ ]:
mean_columns = all_dummy_df.mean()
all_dummy_df = all_dummy_df.fillna(mean_columns)

all_dummy_df.isnull().sum().sort_values(ascending=False).head(10)

## Some analysis


In [ ]:
sns.regplot(x = train['OverallQual'], y = np.exp(Y_train))

In [ ]:
sns.regplot(x = train['YearBuilt'], y = np.exp(Y_train))

In [ ]:
sns.regplot(x = train['OverallCond'], y = np.exp(Y_train))

In [ ]:
sns.regplot(x = train['GrLivArea'], y = np.exp(Y_train))

## Normalize numerical features

Since we are going to do regression on this dataset, it's recommended to standardize our numerical features. So we are going to get all numerical features, subtract by the mean and divide by the standard deviation. This will make them all scaled similarly.

First let's break our dataset into train and test again

In [ ]:
dummy_train_df = all_dummy_df.loc[train.index]
dummy_test_df = all_dummy_df.loc[test.index]

dummy_train_df.shape, dummy_test_df.shape

Now let's get our numerical features

In [ ]:
numeric_cols = all_df.columns[all_df.dtypes != 'object']
numeric_cols

Now we apply the normalization both on our train and test set.

In [ ]:
train_numeric_col_means = dummy_train_df.loc[:, numeric_cols].mean()
train_numeric_col_std = dummy_train_df.loc[:, numeric_cols].std()

dummy_train_df.loc[:, numeric_cols] = (dummy_train_df.loc[:, numeric_cols] - train_numeric_col_means) / train_numeric_col_std

In [ ]:
test_numeric_col_means = dummy_test_df.loc[:, numeric_cols].mean()
test_numeric_col_std = dummy_test_df.loc[:, numeric_cols].std()

dummy_test_df.loc[:, numeric_cols] = (dummy_test_df.loc[:, numeric_cols] - test_numeric_col_means) / test_numeric_col_std

Now let's check the histogram of a variable to see if everything went alright.

In [ ]:
dummy_train_df['GrLivArea'].hist(), dummy_test_df['GrLivArea'].hist()

There are some outliers but we are going to just leave it like that for now.

And this is all the data preparation we need. There's still a lot of things that could be done, but let just proceed for building the model and checking how well we can do with this.


# Building The Model

Let's try building some models now. I'll just try some available in sklearn library and see what works best. 


In [ ]:
all_features = list(dummy_train_df.describe().columns.values)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from operator import itemgetter

Ok, now that we have everything prepared, we are going to follow some steps for each regressor we are going to use.

### Algorithm

1 - find the most relevant/important features for the model\* <br />
2 - tune the hyper-parameters <br />
3 - cross validate the model on our training data 

\*This step will vary depending on the regressor. Tree-based regressor have a built-in feature_importances field which we can use to get the most relevant features. For other approach-based regressors, we are going to use the SelectKBest class to get the most relevant features of our dataset for doing regression on the target.

### 1 - Finding the best features

As mentioned, the way to select features depends on the regressor. Here, we are going to have two approaches, each one defining a function

In [ ]:
# we are going to use this function for tree-based regressors
def get_best_threshold(regressor, X, Y, initial_threshold=0.1, limit_features=100, threshold_increase=-0.0001, title=""):

    base = np.sqrt(-cross_val_score(regressor, X, Y, cv=10, scoring='neg_mean_squared_error'))
    sfm = SelectFromModel(regressor, threshold=initial_threshold)

    n_features = sfm.fit_transform(X, Y).shape[1]
    last_n_features = n_features
    threshold_results = []
    
    f = open("output/{0}.txt".format(title),"w")
    f.write("Result: {0}, N_features: All\n".format(np.mean(base)))
    
    while n_features < limit_features:
        sfm.threshold += threshold_increase
        X_new = sfm.fit_transform(X, Y)
        n_features = X_new.shape[1]

        if n_features > last_n_features:
            
            last_n_features = n_features
            selected_features_score = np.sqrt(-cross_val_score(regressor, X_new, Y, cv=5, scoring='neg_mean_squared_error'))
            f.write("Result: {0}, Threshold: {1}, N_features: {2}\n".format(np.mean(selected_features_score), sfm.threshold, n_features))
            threshold_results.append((sfm.threshold, np.mean(selected_features_score)))
            
    better_threshold = min(threshold_results, key = itemgetter(1))[0]
    f.close()
    return better_threshold

In [ ]:
# we are going to use this function for non-tree-based regressors
def get_best_features(regressor, X, Y, limit_features=100, feature_increase=3, title=""):

    k_features = 10
    k_features_results = []
    f = open("output/{0}.txt".format(title),"w")

    # use SelectKBest to iterate over important features
    while k_features < limit_features:
        selector = SelectKBest(f_regression, k=k_features)
        X_new = selector.fit_transform(X, Y)
        selected_features_score = np.sqrt(-cross_val_score(regressor, X_new, Y, cv=8, scoring='neg_mean_squared_error'))
        f.write("Result: {0}, Variance: {1}, N_features: {2}\n".format(np.mean(selected_features_score),
                                                                       np.var(selected_features_score), k_features))
        k_features_results.append((k_features, np.mean(selected_features_score)))
        k_features += feature_increase
        
    f.close()
    best_k_features = min(k_features_results, key = itemgetter(1))[0]
    return best_k_features

### 2 - Tuning the hyper-parameters

Here we are going to use sklearn's GridSearchCV to exhaustively search a for the best params in a dictionary. You need to pass it values to be tested. If you don't don't know which range of values to use for a parameter, an idea is to check the default value, and work from there. 


In [ ]:
def get_hyper_parameters(alg, parameters, X, Y, title="GridSearch"):
    
    f = open("output/{0}.txt".format(title),"w")
    gs = GridSearchCV(alg, parameters, n_jobs=3, cv=5)                
    gs.fit(X, Y)
    for param in gs.best_params_.keys():
        f.write("Param: {0},  Value: {1}\n".format(param, gs.best_params_[param]))
    f.close()
        
    return gs.best_params_


### 3 - Cross-validating

We don't have any code for this, it will be done after we build the model. One important thing to notice is the number of CV folds to use, and the scorer parameter. Here we are going to use the 'neg_mean_squared_error' because this is the scoring function which Kaggle will validate your results, so it's good to have similar score range while cross-validating, so we can have an idea how well we are going to do on the leaderboard

In [ ]:
# some of these parameters have already been tuned
# we are going to declare them here so we don't have
# grid search everytime we run this notebook

RF_data = {
    #all features                   #selected features
    'n_estimators':       300,     'n_estimators_all':      300,   
    'max_features':       0.5,     'max_features_all':       0.2,  
    'min_samples_leaf':   1,       'min_samples_leaf_all':   1,
    'min_samples_split':  1,       'min_samples_split_all':  4,
    'max_depth':          None,    'max_depth_all':          None,
    'best_threshold':     0.0036
}

GB_data = {
    #all features                  #selected features
    'n_estimators':       300,     'n_estimators_all':       300,    
    'max_features':       'sqrt',  'max_features_all':       'sqrt',     
    'min_samples_leaf':   1,       'min_samples_leaf_all':   1,
    'min_samples_split':  1,       'min_samples_split_all':  1,
    'max_depth':          3,       'max_depth_all':          3,         
    'learning_rate':      0.1,     'learning_rate_all':      0.1, 
    'best_threshold':     0.0024
}

RR_data = {
    #'alpha': 0.568986602902,                   'alpha_all': 0.449843266897,     # normalize = true
    'alpha': 9.5409547635,                      'alpha_all': 15.2641796718,       # normalize = false
    'best_n_features': 156,  
}

EN_data = {
    #'alpha': 0.568986602902,                   'alpha_all': 0.449843266897,     # normalize = true
    'alpha': 9.5409547635,                      'alpha_all': 15.2641796718,       # normalize = false
    'best_n_features': 173,  
}

XGB_data = {
    'n_estimators':       100,
    'learning_rate':      0.1,
    'best_n_features' :   300,
    
}

## Random Forest Regressor

In [ ]:
from sklearn.ensemble.forest import RandomForestRegressor


In [ ]:
alg = RandomForestRegressor(random_state=1)

# First step: get the best features for our train and test
# this step was previously done so we can skip it
#best_threshold = get_best_threshold(alg, dummy_train_df[all_features], Y_train, title="RF_threshold")
#RF_data['best_threshold'] = best_threshold

sfm = SelectFromModel(alg, threshold=RF_data['best_threshold'])
X_train_new = sfm.fit_transform(dummy_train_df[all_features], Y_train)
X_test_new = sfm.transform(dummy_test_df[all_features])

# Second step: grid search for the best params
# this step was previously done so we can skip it
#parameters = {
#    'n_estimators' : [5, 10, 25, 40, 65, 80, 100, 150, 200, 300],
#    'max_features' : ["auto", "sqrt", "log2", 0.2, 0.5],
#    'min_samples_leaf' : [1, 5, 10, 25, 50],
#    'min_samples_split' : [1, 2, 4, 8] ,
#    'max_depth': [None, 3, 5, 10]
#    }

#bp = get_hyper_parameters(alg, parameters, X_train_new, Y_train, "RF_grid_selected")
#bp2 = get_hyper_parameters(alg, parameters, dummy_train_df[all_features], Y_train, "RF_grid_all")
#let's store this information for later use
#RF_data['n_estimators'] = bp['n_estimators']
#RF_data['max_features'] = bp['max_features']
#RF_data['min_samples_split'] = bp['min_samples_split']
#RF_data['min_samples_leaf'] = bp['min_samples_leaf']
#RF_data['max_depth'] = bp['max_depth']

# Third step: cross-validate, here we compare the cross-validated
# scores both using all the features and just the selected features
regr_selected = RandomForestRegressor(random_state=1, n_estimators=RF_data['n_estimators'], max_features=RF_data['max_features'],
                            min_samples_split=RF_data['min_samples_split'], min_samples_leaf=RF_data['min_samples_leaf'],
                            max_depth=RF_data['max_depth'])
regr_all = RandomForestRegressor(random_state=1, n_estimators=RF_data['n_estimators_all'], 
                            max_features=RF_data['max_features_all'], min_samples_split=RF_data['min_samples_split_all'], 
                            min_samples_leaf=RF_data['min_samples_leaf_all'], max_depth=RF_data['max_depth_all'])

selected_features_score = np.sqrt(-cross_val_score(regr_selected, X_train_new, Y_train, cv=8, scoring='neg_mean_squared_error'))
all_features_score = np.sqrt(-cross_val_score(regr_all, dummy_train_df[all_features], Y_train, cv=8, scoring='neg_mean_squared_error'))

selected_features_score = np.mean(selected_features_score)
all_features_score = np.mean(all_features_score)

# Last step: run our model and generate submission files
submission =  pd.DataFrame()
submission["Id"] = test.index

regr_all.fit(dummy_train_df[all_features], Y_train)
predictions = regr_all.predict(dummy_test_df)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/rf_whole_submission.csv", index=False)

regr_selected.fit(X_train_new, Y_train)
predictions = regr_selected.predict(X_test_new)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/rf_subset_submission.csv", index=False)

np.mean(selected_features_score), np.mean(all_features_score)


It's seems that selecting the best features gives us just slightly better results than just using all the generated features. As expected the impact is not so big for tree based approaches, but it should be worth repeating this test for regressors with different approaches.

## Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
alg = GradientBoostingRegressor(random_state=1)

# First step: get the best features for our train and test
# this step was previously done so we can skip it
#best_threshold = get_best_threshold(alg, dummy_train_df[all_features], Y_train, title="GB_threshold")
#GB_data['best_threshold'] = best_threshold

sfm = SelectFromModel(alg, threshold=GB_data['best_threshold'])
X_train_new = sfm.fit_transform(dummy_train_df[all_features], Y_train)
X_test_new = sfm.transform(dummy_test_df[all_features])

# Second step: grid search for the best params
# this step was previously done so we can skip it
#parameters = {
#    'n_estimators' : [5, 10, 25, 40, 65, 80, 100, 150, 200, 300],
#    'max_features' : ["auto", "sqrt", "log2", 0.2, 0.5],
#    'min_samples_leaf' : [1, 5, 10, 25, 50],
#    'min_samples_split' : [1, 2, 4, 8],
#    'max_depth': [None, 3, 5, 10],
#    'learning_rate' : [0.05, 0.08, 0.1, 0.15, 0.2]
#    }
#
#bp = get_hyper_parameters(alg, parameters, X_train_new, Y_train, "GB_grid_selected")
#bp = get_hyper_parameters(alg, parameters, X_train_new, Y_train, "GB_grid_all")
#let's store this information for later use
#GB_data['n_estimators'] = bp['n_estimators']
#GB_data['max_features'] = bp['max_features']
#GB_data['min_samples_split'] = bp['min_samples_split']
#GB_data['min_samples_leaf'] = bp['min_samples_leaf']
#GB_data['max_depth'] = bp['max_depth']
#GB_data['learning_rate'] = bp['learning_rate']

# Third step: cross-validate, here we compare the cross-validated
# scores both using all the features and just the selected features
regr_selected = GradientBoostingRegressor(random_state=1, n_estimators=GB_data['n_estimators'], 
                                max_features=GB_data['max_features'], min_samples_split=GB_data['min_samples_split'], 
                                min_samples_leaf=GB_data['min_samples_leaf'], learning_rate=GB_data['learning_rate'])
regr_all = GradientBoostingRegressor(random_state=1, n_estimators=GB_data['n_estimators_all'], 
                                max_features=GB_data['max_features_all'], 
                                min_samples_split=GB_data['min_samples_split_all'], 
                                min_samples_leaf=GB_data['min_samples_leaf_all'], 
                                learning_rate=GB_data['learning_rate_all'])

selected_features_score = np.sqrt(-cross_val_score(regr_selected, X_train_new, Y_train, cv=8, scoring='neg_mean_squared_error'))
all_features_score = np.sqrt(-cross_val_score(regr_all, dummy_train_df[all_features], Y_train, cv=8, scoring='neg_mean_squared_error'))

# Last step: run our model and generate submission files
submission =  pd.DataFrame()
submission["Id"] = test.index

regr_all.fit(dummy_train_df[all_features], Y_train)
predictions = regr_all.predict(dummy_test_df)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/gb_whole_submission.csv", index=False)

regr_selected.fit(X_train_new, Y_train)
predictions = regr_selected.predict(X_test_new)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/gb_subset_submission.csv", index=False)

np.mean(selected_features_score), np.mean(all_features_score)


## Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
alg = Ridge(random_state=1)

# First step: get the best features for our train and test
# this step was previously done so we can skip it
#best_n_features = get_best_features(alg, dummy_train_df[all_features], Y_train, len(all_features)+1, 1, title="RR_threshold")
#RR_data['best_n_features'] = best_n_features

selector = SelectKBest(f_regression, k=RR_data['best_n_features'])
X_train_new = selector.fit_transform(dummy_train_df[all_features], Y_train)
X_test_new = selector.transform(dummy_test_df[all_features])

# Second step: grid search for the best params
# this step was previously done so we can skip it
parameters = {
'alpha' : np.logspace(-3, 2, 50),
}
#
RR_data['alpha'] = get_hyper_parameters(alg, parameters, X_train_new, Y_train, "RR_grid_selected")['alpha']
RR_data['alpha_all'] = get_hyper_parameters(alg, parameters, dummy_train_df[all_features], Y_train, "RR_grid_all")['alpha']

# Third step: cross-validate, here we compare the cross-validated
# scores both using all the features and just the selected features
regr_selected = Ridge(random_state=1, alpha=RR_data['alpha'])
regr_all = Ridge(random_state=1, alpha=RR_data['alpha_all'])
          
selected_features_score = np.sqrt(-cross_val_score(regr_selected, X_train_new, Y_train, cv=8, scoring='neg_mean_squared_error'))
all_features_score = np.sqrt(-cross_val_score(regr_all, dummy_train_df[all_features], Y_train, cv=8, scoring='neg_mean_squared_error'))

# Last step: run our model and generate submission files
submission =  pd.DataFrame()
submission["Id"] = test.index

regr_all.fit(dummy_train_df[all_features], Y_train)
predictions = regr_all.predict(dummy_test_df)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/ridge_whole_submission.csv", index=False)

regr_selected.fit(X_train_new, Y_train)
predictions = regr_selected.predict(X_test_new)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/ridge_subset_submission.csv", index=False)

print(RR_data['best_n_features'])
np.mean(selected_features_score), np.mean(all_features_score)

## Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
alg = ElasticNet(random_state=1,  alpha=EN_data['alpha'], normalize=True)

# First step: get the best features for our train and test
# this step was previously done so we can skip it
#best_n_features = get_best_features(alg, dummy_train_df[all_features], Y_train, len(all_features)+1, 1, title="EN_threshold")
#EN_data['best_n_features'] = best_n_features

selector = SelectKBest(f_regression, k=EN_data['best_n_features'])
X_train_new = selector.fit_transform(dummy_train_df[all_features], Y_train)
X_test_new = selector.transform(dummy_test_df[all_features])

# Second step: grid search for the best params
# this step was previously done so we can skip it
parameters = {
'alpha' : np.logspace(-3, 2, 50),
'l1_ratio' : np.arange(0.5,0.8,0.1)
}
#
bp1 = get_hyper_parameters(alg, parameters, X_train_new, Y_train, "EN_grid_selected")
bp2 = get_hyper_parameters(alg, parameters, dummy_train_df[all_features], Y_train, "EN_grid_all")
EN_data['alpha'] = bp1['alpha']
EN_data['alpha_all'] = bp2['alpha']
EN_data['l1_ratio'] = bp1['l1_ratio']
EN_data['l1_ratio_all'] = bp2['l1_ratio']

# Third step: cross-validate, here we compare the cross-validated
# scores both using all the features and just the selected features
regr_selected = ElasticNet(random_state=1, alpha=EN_data['alpha'], l1_ratio=EN_data['l1_ratio'], normalize=True)
regr_all = ElasticNet(random_state=1, alpha=EN_data['alpha_all'], l1_ratio=EN_data['l1_ratio_all'],normalize=True)
          
selected_features_score = np.sqrt(-cross_val_score(regr_selected, X_train_new, Y_train, cv=50, scoring='neg_mean_squared_error'))
all_features_score = np.sqrt(-cross_val_score(regr_all, dummy_train_df[all_features], Y_train, cv=50, scoring='neg_mean_squared_error'))

# Last step: run our model and generate submission files
submission =  pd.DataFrame()
submission["Id"] = test.index

regr_all.fit(dummy_train_df[all_features], Y_train)
predictions = regr_all.predict(dummy_test_df)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/elasticnet_whole_submission.csv", index=False)

regr_selected.fit(X_train_new, Y_train)
predictions = regr_selected.predict(X_test_new)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/elasticnet_subset_submission.csv", index=False)

print(EN_data['best_n_features'])
print(EN_data['l1_ratio'])
print(EN_data['l1_ratio_all'])
np.mean(selected_features_score), np.mean(all_features_score)

## Extreme Gradient Boosting

In [ ]:
import xgboost as xgb

In [ ]:
alg = xgb.XGBRegressor(seed=0, nthread=3, n_estimators=XGB_data['n_estimators'],
                                learning_rate=XGB_data['learning_rate'])

# First step: get the best features for our train and test
# this step was previously done so we can skip it
#best_threshold = get_best_features(alg, dummy_train_df[all_features], Y_train, len(all_features)+1, title="XGB_threshold")
#XGB_data['best_threshold'] = best_threshold

selector = SelectKBest(f_regression, k=XGB_data['best_n_features'])
X_train_new = selector.fit_transform(dummy_train_df[all_features], Y_train)
X_test_new = selector.transform(dummy_test_df[all_features])

# Second step: grid search for the best params
# this step was previously done so we can skip it
#parameters = {
#    'n_estimators' : [5, 10, 25, 40, 65, 80, 100, 150, 200, 300],
#    'min_samples_leaf' : [1, 5, 10, 25, 50],
#    'min_samples_split' : [1, 2, 4, 8],
#    'max_depth': [None, 3, 5, 10],
#    'learning_rate' : [0.05, 0.08, 0.1, 0.15, 0.2]
#    }
#
#bp = get_hyper_parameters(alg, parameters, X_train_new, Y_train, "XGB_grid_selected")
#bp = get_hyper_parameters(alg, parameters,  dummy_train_df[all_features], Y_train, "XGB_grid_all")

# Third step: cross-validate, here we compare the cross-validated
# scores both using all the features and just the selected features
regr = xgb.XGBRegressor(seed=0, nthread=3, n_estimators=XGB_data['n_estimators'],
                                learning_rate=XGB_data['learning_rate'])

selected_features_score = np.sqrt(-cross_val_score(regr, X_train_new, Y_train, cv=8, scoring='neg_mean_squared_error'))
all_features_score = np.sqrt(-cross_val_score(regr, dummy_train_df[all_features], Y_train, cv=8, scoring='neg_mean_squared_error'))

# Last step: run our model and generate submission files
submission =  pd.DataFrame()
submission["Id"] = test.index

regr.fit(dummy_train_df[all_features], Y_train)
predictions = regr.predict(dummy_test_df)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/xgb_whole_submission.csv", index=False)

regr.fit(X_train_new, Y_train)
predictions = regr.predict(X_test_new)
submission["SalePrice"] = np.exp(predictions)
submission.to_csv("output/xgb_subset_submission.csv", index=False)

np.mean(selected_features_score), np.mean(all_features_score)

## Multi-layer Perceptron Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
#alg = MLPRegressor(random_state=1)
#
#best_features = get_best_features(alg, dummy_train_df[all_features], Y_train, 100, 3)
#          
#selector = SelectKBest(f_regression, k=best_features)
#X_train_new = selector.fit_transform(dummy_train_df[all_features], Y_train)
#X_test_new = selector.transform(dummy_test_df[all_features])
#          
#now let's try comparing the selected KBest features with all features
#selected_features_score = np.sqrt(-cross_val_score(alg, X_train_new, Y_train, cv=3, scoring='neg_mean_squared_error'))
#all_features_score = np.sqrt(-cross_val_score(alg, dummy_train_df[all_features], Y_train, cv=3, scoring='neg_mean_squared_error'))
#
#np.mean(selected_features_score), np.mean(all_features_score)

For the Multi-Layer Perceptron it seems that having many features just make it worse. Using only the overall quality of the house as a feature wields the best result, which is far from good by the way. For now, let's just be done with this model.

## Support Vector Machine Regressor

In [ ]:
from sklearn.svm import SVR

In [ ]:
#alg = SVR()
#alg = SVR(epsilon=0, C=2, gamma=3.0517578125e-05)
#
#best_features = get_best_features(alg, dummy_train_df[all_features], Y_train, 100, 3)
#          
#selector = SelectKBest(f_regression, k=best_features)
#X_train_new = selector.fit_transform(dummy_train_df[all_features], Y_train)
#X_test_new = selector.transform(dummy_test_df[all_features])
#
#parameters = {   
#    'C' : [2**i for i in range(-5,15, 2)], # 2^-5, 2^-3,..., 2^15
#    'gamma' : [2**i for i in range(-15,3, 2)], # 2^-15, 2^-13,..., 2^3
#    'epsilon' : [0, 0.01, 0.1, 0.5, 1, 2, 4]
#}    
#
#bp = get_hyper_parameters(alg, parameters, X_train_new, Y_train)
#alg = SVR(epsilon=bp['epsilon'], C=bp['C'], gamma=bp['gamma'])
#    
#now let's try comparing the selected KBest features with all features
#selected_features_score = np.sqrt(-cross_val_score(alg, X_train_new, Y_train, cv=3, scoring='neg_mean_squared_error'))
#all_features_score = np.sqrt(-cross_val_score(alg, dummy_train_df[all_features], Y_train, cv=3, scoring='neg_mean_squared_error'))
#
#np.mean(selected_features_score), np.mean(all_features_score)

Same reasoning for MLP goes for SVR, using many features is not very helpful.

## Ensemble

In [ ]:
#we will pass a list of regressors and this function 
# will perform the ensemble for us
def perform_ensemble(regressors, train, target, test, subset_size=0.8):
    total_predictions = []
    total_scores = []
    seeds = []
    for alg in regressors:
        train["SalePrice"] = target
        random_state = alg.get_params()["random_state"]
        subset = train.sample(frac=subset_size, random_state=random_state)
        subset_target = subset.pop("SalePrice")
        
        score = np.sqrt(-cross_val_score(alg, subset, subset_target, cv=8, scoring='neg_mean_squared_error'))
        if not total_scores or np.mean(score) < np.mean(total_scores):
            total_scores.append(score)
            print("{0}: Mean: {1} (New score: {2}, seed: {3})".format(len(total_predictions), 
                                                        np.mean(total_scores), np.mean(score), random_state))
        
            alg.fit(subset, subset_target)
            prediction = alg.predict(test)
            total_predictions.append(prediction)

    # average scores
    return np.mean(total_predictions, axis=0)


regressors = []
for i in range(20):
    seed = i #np.random.randint(4294967295)
    regressors.append(Ridge(random_state=seed, alpha=RR_data['alpha_all']))
#regressors.append(Ridge(random_state=1, alpha=RR_data['alpha_all']))
#regressors.append(RandomForestRegressor(random_state=1, n_estimators=RF_data['n_estimators_all'], 
#                            max_features=RF_data['max_features_all'], min_samples_split=RF_data['min_samples_split_all'], 
#                            min_samples_leaf=RF_data['min_samples_leaf_all'], max_depth=RF_data['max_depth_all']))
#regressors.append(GradientBoostingRegressor(random_state=1, n_estimators=GB_data['n_estimators_all'], 
#                                max_features=GB_data['max_features_all'], 
#                                min_samples_split=GB_data['min_samples_split_all'], 
#                                min_samples_leaf=GB_data['min_samples_leaf_all'], 
#                                learning_rate=GB_data['learning_rate_all']))
result = perform_ensemble(regressors, dummy_train_df[all_features], Y_train, dummy_test_df)

submission =  pd.DataFrame()
submission["Id"] = test.index

submission["SalePrice"] = np.exp(result)
submission.to_csv("output/ensemble_submission.csv", index=False)

## Reference Links

[Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project (read this for insights on the dataset)](https://ww2.amstat.org/publications/jse/v19n3/decock.pdf)

[How to Tune Algorithm Parameters with Scikit-Learn](http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/)

[Parameter Tuning in Gradient Boosting (GBM) in Python](https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/)

[Ridge and Lasso tutorial](https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/)

[Tuning the parameters of your Random Forest model](https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/)

[Ensemble Modeling: Stack Model Example](https://www.kaggle.com/jimthompson/house-prices-advanced-regression-techniques/ensemble-model-stacked-model-example/discussion)